https://medium.com/tensorflow/building-a-text-classification-model-with-tensorflow-hub-and-estimators-3169e7aa568

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.read_json("batched3.json")

In [3]:
df = df.sample(frac=1).reset_index(drop=True)

In [4]:
df.head()

,text,categories
0,The pioneering label Stax Records is best know...,"[News, NYTimes]"
1,[WP] Your ability to see what level of pain a ...,"[Reddit, Short Story]"
2,CAIRO ā€” Jordanā€™s government on Tuesday bar...,"[News, NYTimes]"
3,"Criminal Her skin was pale, and as beautiful a...","[Reddit, Short Story]"
4,[WP] You are a member of an elite squad tasked...,"[Reddit, Short Story]"


In [5]:
text = df['text']
categories = df['categories']

set1 = categories.apply(lambda x: x[0])
set2 = categories.apply(lambda x: x[1])

set1 = set1.unique()
set2 = set2.unique()

unique_categories = []

for s in set1:
    unique_categories.append(s)
for s in set2:
    unique_categories.append(s)

unique_categories

In [6]:
train_size = int(len(text) * .8)

train_text = text[:train_size]
train_categories = categories[:train_size]

test_text = text[train_size:]
test_categories = categories[train_size:]

In [7]:
text_embeddings = hub.text_embedding_column(
  "pieces_of_text", 
  module_spec="https://tfhub.dev/google/universal-sentence-encoder/2"
)

In [8]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_categories)
train_encoded = encoder.transform(train_categories)
test_encoded = encoder.transform(test_categories)
num_classes = len(encoder.classes_)

In [9]:
encoder.classes_

array(['NYTimes', 'Negative Tweet', 'News', 'Positive Tweet', 'Reddit',
       'Short Story', 'Twitter'], dtype=object)

In [10]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [11]:
estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[text_embeddings]
)

INFO:tensorflow:Using default config.


I0429 13:42:50.794952 10908 estimator.py:1739] Using default config.


W0429 13:42:50.797950 10908 estimator.py:1760] Using temporary folder as model directory: C:\Users\Bruger\AppData\Local\Temp\tmpedyakgb7


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Bruger\\AppData\\Local\\Temp\\tmpedyakgb7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002DE2AB71630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0429 13:42:50.802934 10908 estimator.py:201] Using config: {'_model_dir': 'C:\\Users\\Bruger\\AppData\\Local\\Temp\\tmpedyakgb7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002DE2AB71630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
# Format our data for the numpy_input_fn
features = {
  "pieces_of_text": np.array(train_text)
}
labels = np.array(train_encoded)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    features, 
    labels, 
    shuffle=True, 
    batch_size=32, 
    num_epochs=20
)

In [13]:
estimator.train(input_fn=train_input_fn)

Instructions for updating:
Colocations handled automatically by placer.


W0429 13:42:50.858436 10908 deprecation.py:323] From C:\Users\Bruger\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


W0429 13:42:50.877386 10908 deprecation.py:323] From C:\Users\Bruger\Anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\inputs\queues\feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

INFO:tensorflow:global_step/sec: 4.20081


I0429 13:44:01.803238 10908 basic_session_run_hooks.py:680] global_step/sec: 4.20081


INFO:tensorflow:loss = 0.61571, step = 101 (23.802 sec)


I0429 13:44:01.808224 10908 basic_session_run_hooks.py:247] loss = 0.61571, step = 101 (23.802 sec)


KeyboardInterrupt: 

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"pieces_of_text": np.array(test_text).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)

estimator.evaluate(input_fn=eval_input_fn)

In [ ]:
raw_test = ["In a land, not so far from this one, there was a little village. This village was known as waffle-town. In this quiet town lived a fellow named Waffle lad. Waffle lad had lived a happy, albeit boring life up to this point. The world was magical, but you become accustomed to the magic over time. The rivers of maple syrup, the trees that dripped butter for sap. It was wonderful but waffle lad was an adventurous soul. He hungered and yearned for the great unknown.",
           "However the curse had already begun. To punish waffle lad for his gluttony, pancake Sam had cursed the fruit. Any who consume the fruit would have their body swell to unbelievable size, and never be able to consume food again, until eventually they died from asphyxiation or hunger. Waffle lad of course had no idea, and simply carried on his daily life. Boring, but safe. Waffle lad remembered fondly the thrill of adventure and risk as he picked peanuts."]

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"pieces_of_text": np.array(raw_test).astype(np.str)}, shuffle=False)

results = estimator.predict(predict_input_fn)

In [ ]:
results

In [ ]:
for categories in results:
  top_2 = categories['probabilities'].argsort()[-2:][::-1]
  for category in top_2:
    text_category = encoder.classes_[category]
    print(text_category + ': ' + str(round(categories['probabilities'][category] * 100, 2)) + '%')
  printline()